In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import Series
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
font_path = 'AppleGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=18)
font_name = fm.FontProperties(fname=font_path).get_name()

In [ ]:
food_waste = pd.read_csv("01_음식물쓰레기_FOOD_WASTE_210811_update.CSV", encoding="cp949")
card_spending = pd.read_csv("04_음식관련 카드소비_CARD_SPENDING.CSV", encoding="cp949")

In [ ]:
cs = card_spending.copy()
#다운샘플링
def get_year_month(x):
    return x[0:7]
cs['year_month'] = cs['base_date'].apply(lambda x: get_year_month(x))
cs= cs.drop(['base_date'], axis=1, inplace=False)

In [40]:
cs_1 = cs.groupby(['year_month','emd_nm', 'mct_cat_nm', 'mct_cat_cd']).sum().reset_index()
cs_1

,year_month,emd_nm,mct_cat_nm,mct_cat_cd,use_cnt,use_amt
0,2018-01,건입동,간식,C00400,4540,52714107
1,2018-01,건입동,농축수산물,C01200,1699,180535251
2,2018-01,건입동,마트/슈퍼마켓,A00200,17547,613458055
3,2018-01,건입동,배달,C01400,585,11571838
4,2018-01,건입동,부페,C00700,55,5081850
...,...,...,...,...,...,...
20000,2021-06,효돈동,아시아음식,C00300,720,25716378
20001,2021-06,효돈동,양식,C00200,225,9302910
20002,2021-06,효돈동,주점및주류판매,C00900,105,4039593
20003,2021-06,효돈동,패스트푸드,C00500,1205,20827922


In [5]:
fw = food_waste.copy()

def get_year_month(x):
    return x[0:7]
fw['year_month'] = fw['base_date'].apply(lambda x: get_year_month(x))

In [6]:
fw = fw.drop(['base_date'], axis=1, inplace=False)

In [39]:
fw_1 = fw.groupby(['year_month','emd_nm']).sum().reset_index()
fw_1

,year_month,emd_nm,em_cnt,em_g,pay_amt
0,2018-01,건입동,18416,45646550,1364804
1,2018-01,남원읍,9570,42437700,1270773
2,2018-01,노형동,113990,265018400,7883340
3,2018-01,대륜동,21666,57612600,1676850
4,2018-01,대정읍,10185,38885550,1164122
...,...,...,...,...,...
1661,2021-06,표선면,10751,42453750,1271419
1662,2021-06,한경면,8031,27060150,809898
1663,2021-06,한림읍,25653,82746990,2476292
1664,2021-06,화북동,66088,110750050,3306029


In [41]:
temp_1 = pd.merge(cs_1, fw_1, on=['year_month', 'emd_nm'])
temp_1
# 결측값 없이 두 데이터셋 merge

,year_month,emd_nm,mct_cat_nm,mct_cat_cd,use_cnt,use_amt,em_cnt,em_g,pay_amt
0,2018-01,건입동,간식,C00400,4540,52714107,18416,45646550,1364804
1,2018-01,건입동,농축수산물,C01200,1699,180535251,18416,45646550,1364804
2,2018-01,건입동,마트/슈퍼마켓,A00200,17547,613458055,18416,45646550,1364804
3,2018-01,건입동,배달,C01400,585,11571838,18416,45646550,1364804
4,2018-01,건입동,부페,C00700,55,5081850,18416,45646550,1364804
...,...,...,...,...,...,...,...,...,...
18078,2021-06,효돈동,아시아음식,C00300,720,25716378,8720,27554100,824457
18079,2021-06,효돈동,양식,C00200,225,9302910,8720,27554100,824457
18080,2021-06,효돈동,주점및주류판매,C00900,105,4039593,8720,27554100,824457
18081,2021-06,효돈동,패스트푸드,C00500,1205,20827922,8720,27554100,824457


In [29]:
#행정동별, 카테고리별 소비총액의 lag(1)과 음식물쓰레기 배출량 상관관계
corr_data_1 = pd.DataFrame(index=range(0,len(set(temp_1['emd_nm']))),columns = ['emd_nm']+list(set(temp_1['mct_cat_nm'])))
n = 0
for i in tqdm(set(temp_1['emd_nm'])):
    print(i)
    temp_list = [np.nan]
    temp1 = temp_1[temp_1['emd_nm']==i]
    temp2 = temp_2[temp_1['emd_nm']==i]
    for j in set(temp_1['mct_cat_nm']):
        temp_22 = temp_1[temp_1['mct_cat_nm']==j]
        temp = temp2
        temp['em_g'] = temp['em_g'].shift(1)
        temp['em_g']= temp['em_g'].diff()
        temp['use_amt']= temp['use_amt'].diff()
        temp['use_amt']= temp['use_amt'].shift(1)
        temp = temp.dropna(axis=0)
        temp_corr = temp[['em_g', 'use_amt']].corr()
        temp_list.append(round(temp_corr.iloc[0,1],2))
    corr_data_1.iloc[n,:] = temp_list
    n += 1
    
corr_data_1['emd_nm'] = set(temp_1['emd_nm'])
corr_data_1 = corr_data_1.set_index(['emd_nm'])


  0%|          | 0/41 [00:00<?, ?it/s]/home/yoonjeong.won/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/yoonjeong.won/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/home/yoonjeong.won/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/home/yoonjeong.

조천읍
서홍동
표선면


 10%|▉         | 4/41 [00:00<00:02, 13.34it/s]

오라동
영천동
삼도2동


 20%|█▉        | 8/41 [00:00<00:02, 13.82it/s]

...

안덕면
아라동
구좌읍


 93%|█████████▎| 38/41 [00:02<00:00, 14.02it/s]

외도동
일도1동
용담2동


100%|██████████| 41/41 [00:02<00:00, 14.02it/s]

송산동
한림읍


In [ ]:
corr_data_1 = pd.DataFrame(index=range(0,len(set(temp7['emd_nm']))),columns = ['emd_nm']+list(set(temp7['mct_cat_nm'])))
n = 0
for i in tqdm(set(temp7['emd_nm'])):
    print(i)
    temp_list = [np.nan]
    temp_1 = temp7[temp7['emd_nm']==i]
    for j in set(temp5['mct_cat_nm']):
        temp_2 = temp7[temp7['mct_cat_nm']==j]
        temp = pd.merge(temp_1[['year_month', 'em_g', 'emd_cd','emd_nm']], temp_2[['year_month', 'mct_cat_nm','ratio','emd_cd']], how='left')
        temp = temp.dropna(axis=0)
        temp_corr = temp[['em_g', 'ratio']].corr()
        print(round(temp_corr.iloc[0,1],2))
        temp_list.append(round(temp_corr.iloc[0,1],2))
    corr_data_1.iloc[n,:] = temp_list
    n += 1
corr_data_1['emd_nm'] = set(temp7['emd_nm'])

In [13]:
# 카테고리별 배출량과 카드소비량 상관관계 lag(1), diff()
for i in sorted(set(temp_2['mct_cat_nm'])):
    print(i)
    temp_list = [np.nan]
    temp = temp_2[temp_2['mct_cat_nm']==i]
    temp['em_g_d1']= temp['em_g'].diff()
    temp['em_g_d1']= temp['em_g_d1'].shift(1)
    temp['use_amt_d1']= temp['use_amt'].diff()
    temp['use_amt_d1']= temp['use_amt_d1'].shift(1)
    temp= temp.dropna()
    
    temp_corr = temp[['em_g_d1', 'use_amt_d1']].corr()
    temp_list.append(round(temp_corr.iloc[0,1],2))

간식
농축수산물
마트/슈퍼마켓
배달
부페
식품
아시아음식
양식
주점및주류판매
패스트푸드
한식


/home/yoonjeong.won/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/yoonjeong.won/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/yoonjeong.won/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

# 클러스터별, 카테고리별, 배출량과 카드소비량 상관관계 분석 lag(1), diff()

In [21]:
cluster = pd.read_csv('클러스터링 최종.csv')

In [44]:
cl= cluster.drop(['관광인구','거주인구','spot'], axis=1, inplace=False)
cl.head()

,emd_cd,emd_nm,cluster
0,50110590,건입동,7
1,50110256,구좌읍,5
2,50130253,남원읍,7
3,50110660,노형동,3
4,50130590,대륜동,9


In [42]:
temp_1

,year_month,emd_nm,mct_cat_nm,mct_cat_cd,use_cnt,use_amt,em_cnt,em_g,pay_amt
0,2018-01,건입동,간식,C00400,4540,52714107,18416,45646550,1364804
1,2018-01,건입동,농축수산물,C01200,1699,180535251,18416,45646550,1364804
2,2018-01,건입동,마트/슈퍼마켓,A00200,17547,613458055,18416,45646550,1364804
3,2018-01,건입동,배달,C01400,585,11571838,18416,45646550,1364804
4,2018-01,건입동,부페,C00700,55,5081850,18416,45646550,1364804
...,...,...,...,...,...,...,...,...,...
18078,2021-06,효돈동,아시아음식,C00300,720,25716378,8720,27554100,824457
18079,2021-06,효돈동,양식,C00200,225,9302910,8720,27554100,824457
18080,2021-06,효돈동,주점및주류판매,C00900,105,4039593,8720,27554100,824457
18081,2021-06,효돈동,패스트푸드,C00500,1205,20827922,8720,27554100,824457


In [23]:
cluster.rename(columns={'행정동코드':'emd_cd'}, inplace=True)
cluster.rename(columns={'행정동명':'emd_nm'}, inplace=True)

In [45]:
df = pd.merge(temp_1, cl, on=['emd_nm'])
df

,year_month,emd_nm,mct_cat_nm,mct_cat_cd,use_cnt,use_amt,em_cnt,em_g,pay_amt,emd_cd,cluster
0,2018-01,건입동,간식,C00400,4540,52714107,18416,45646550,1364804,50110590,7
1,2018-01,건입동,농축수산물,C01200,1699,180535251,18416,45646550,1364804,50110590,7
2,2018-01,건입동,마트/슈퍼마켓,A00200,17547,613458055,18416,45646550,1364804,50110590,7
3,2018-01,건입동,배달,C01400,585,11571838,18416,45646550,1364804,50110590,7
4,2018-01,건입동,부페,C00700,55,5081850,18416,45646550,1364804,50110590,7
...,...,...,...,...,...,...,...,...,...,...,...
17751,2021-06,한림읍,아시아음식,C00300,3190,105470802,25653,82746990,2476292,50110250,8
17752,2021-06,한림읍,양식,C00200,1327,40153504,25653,82746990,2476292,50110250,8
17753,2021-06,한림읍,주점및주류판매,C00900,170,9104552,25653,82746990,2476292,50110250,8
17754,2021-06,한림읍,패스트푸드,C00500,4315,81433739,25653,82746990,2476292,50110250,8


In [53]:
temp= df[df['cluster']==1]
temp = temp.groupby(['year_month', 'mct_cat_nm','cluster']).sum().reset_index()
temp

,year_month,mct_cat_nm,cluster,use_cnt,use_amt,em_cnt,em_g,pay_amt,emd_cd
0,2018-01,간식,1,52496,610608163,209431,516780600,15451299,300663350
1,2018-01,농축수산물,1,20416,1286850315,209431,516780600,15451299,300663350
2,2018-01,마트/슈퍼마켓,1,216880,6941575601,209431,516780600,15451299,300663350
3,2018-01,배달,1,5540,109035309,209431,516780600,15451299,300663350
4,2018-01,부페,1,489,34966074,209431,516780600,15451299,300663350
...,...,...,...,...,...,...,...,...,...
457,2021-06,아시아음식,1,20074,652042030,250606,442458300,13211344,300663350
458,2021-06,양식,1,7422,229108605,250606,442458300,13211344,300663350
459,2021-06,주점및주류판매,1,2815,108391219,250606,442458300,13211344,300663350
460,2021-06,패스트푸드,1,31610,460075391,250606,442458300,13211344,300663350


In [ ]:
for i in range(1,10):
    print(i)
    temp1 = df[df['cluster']==i]
    temp1= temp1.groupby(['year_month', 'mct_cat_nm','cluster']).sum().reset_index()
    
    for j in sorted(set(temp1['mct_cat_nm'])):
    print(j)
    temp2_list = [np.nan]
    temp2 = temp1[temp1['mct_cat_nm']==i]
    temp2['em_g']= temp2['em_g'].diff()
    temp2['em_g']= temp2['em_g'].shift(1)
    temp2['use_amt']= temp2['use_amt'].diff()
    temp2['use_amt']= temp2['use_amt'].shift(1)
    temp= temp.dropna(axis=0)
    temp_corr = temp2[['em_g', 'use_amt']].corr()
    print(round(temp_corr.iloc[0,1],2))
    temp2_list.append(round(temp_corr.iloc[0,1],2))